<a href="https://colab.research.google.com/github/linkjavier/disco/blob/felipe/felipe/felipe_music_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From book: Generative AI with Python and TensorFlow 2
By Joseph Babcock and Raghav Bali

LSTM-Attention Model for Music Generation

In [ ]:
import os
import pickle
import numpy as np
from music21 import note, chord, corpus, converter, stream, instrument
import glob
from tensorflow.keras.utils import to_categorical
from tqdm.notebook import tqdm

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
!unzip '/content/drive/MyDrive/midi_dataset.zip'

In [ ]:
data_dir = 'midi_dataset'

# list of files
midi_list = os.listdir(data_dir)

# Load and make list of stream objects
original_scores = []
for midi in tqdm(midi_list):
    score = converter.parse(os.path.join(data_dir,midi))
    original_scores.append(score)

  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
# Merge notes into chords
original_scores = [midi.chordify() for midi in tqdm(original_scores)]

  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
# Define empty lists of lists
original_chords = [[] for _ in original_scores]
original_durations = [[] for _ in original_scores]
original_keys = []

# Extract notes, chords, durations, and keys
for i, midi in tqdm(enumerate(original_scores)):
    original_keys.append(str(midi.analyze('key')))
    for element in midi:
        if isinstance(element, note.Note):
            original_chords[i].append(element.pitch)
            original_durations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            original_chords[i].append('.'.join(str(n) for n in element.pitches))
            original_durations[i].append(element.duration.quarterLength)

0it [00:00, ?it/s]

In [ ]:
# Create list of chords and durations from songs in C major
major_chords = [c for (c, k) in tqdm(zip(original_chords, original_keys)) if (k == 'C major')]
major_durations = [c for (c, k) in tqdm(zip(original_durations, original_keys)) if (k == 'C major')]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)

def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = dict((element, number) for number, element in enumerate(element_names))
    int_to_element = dict((number, element) for number, element in enumerate(element_names))

    return (element_to_int, int_to_element)

In [ ]:
store_folder = 'parsed_data'
os.mkdir(store_folder)

In [ ]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct([n for chord in major_chords for n in chord])
duration_names, n_durations = get_distinct([d for dur in major_durations for d in dur])
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

In [ ]:
# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [ ]:
# Set sequence length
sequence_length = 64

# Define empty array for train data
train_chords = []
train_durations = []
target_chords = []
target_durations = []

# Construct train and target sequences for chords and durations
for s in range(len(major_chords)):
    chord_list = [note_to_int[c] for c in major_chords[s]]
    duration_list = [duration_to_int[d] for d in major_durations[s]]
    for i in range(len(chord_list) - sequence_length):
        train_chords.append(chord_list[i:i+sequence_length])
        train_durations.append(duration_list[i:i+sequence_length])
        target_chords.append(chord_list[i+1])
        target_durations.append(duration_list[i+1])

In [ ]:
train_chords = np.array(train_chords)
train_durations = np.array(train_durations)
target_chords = np.array(target_chords)
target_durations = np.array(target_durations)

In [ ]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):
    """ create the structure of the neural network """

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])

    x = LSTM(rnn_units, return_sequences=True)(x)

    x = LSTM(rnn_units, return_sequences=True)(x)

    # attention
    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)
    alpha = Activation('softmax')(e)

    alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

    c = Multiply()([x, alpha_repeated])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])
    model.compile(loss=['sparse_categorical_crossentropy', 
                        'sparse_categorical_crossentropy'], optimizer=RMSprop(learning_rate = 0.001))

    return model

In [ ]:
embed_size = 128
rnn_units = 128

In [ ]:
model = create_network(n_notes, n_durations, embed_size, rnn_units)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 128)    379264      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 128)    2304        input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
os.mkdir('outputs')
os.mkdir('outputs/output')
os.mkdir('outputs/weights')

In [ ]:
output_folder = 'outputs'
weights_folder = os.path.join(output_folder, 'weights')


checkpoint = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

epoch_checkpoint = ModelCheckpoint(
    os.path.join(weights_folder, "weights-{epoch:02d}-{loss:.4f}.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 5
)


callbacks_list = [
    checkpoint
    , epoch_checkpoint
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))

In [ ]:
history = model.fit([train_chords, train_durations], 
                    [target_chords, target_durations]
                    , epochs=100, batch_size=128
                    , callbacks=callbacks_list
                    , shuffle=True
                  )

Epoch 1/100
92/92 [==============================] - 12s 41ms/step - loss: 8.4728 - pitch_loss: 7.3343 - duration_loss: 1.1385
Epoch 2/100
 1/92 [..............................] - ETA: 3s - loss: 7.7201 - pitch_loss: 6.8737 - duration_loss: 0.8464

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


92/92 [==============================] - 4s 40ms/step - loss: 7.5568 - pitch_loss: 6.7380 - duration_loss: 0.8189
Epoch 3/100
92/92 [==============================] - 4s 40ms/step - loss: 7.2037 - pitch_loss: 6.4726 - duration_loss: 0.7311
Epoch 4/100
92/92 [==============================] - 4s 40ms/step - loss: 6.9700 - pitch_loss: 6.3118 - duration_loss: 0.6583
Epoch 5/100
92/92 [==============================] - 4s 40ms/step - loss: 6.7818 - pitch_loss: 6.1709 - duration_loss: 0.6109
Epoch 6/100
92/92 [==============================] - 4s 40ms/step - loss: 6.6196 - pitch_loss: 6.0422 - duration_loss: 0.5774
Epoch 7/100
92/92 [==============================] - 4s 40ms/step - loss: 6.4574 - pitch_loss: 5.9134 - duration_loss: 0.5440
Epoch 8/100
92/92 [==============================] - 4s 40ms/step - loss: 6.2965 - pitch_loss: 5.7809 - duration_loss: 0.5156
Epoch 9/100
92/92 [==============================] - 4s 40ms/step - loss: 6.1755 - pitch_loss: 5.6767 - duration_loss: 0.4988
Epoc

In [ ]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [ ]:
# chord and duration sequences
initial_chords = np.expand_dims(train_chords[0,:].copy(), 0)
initial_durations = np.expand_dims(train_durations[0,:].copy(), 0)

In [ ]:
initial_chords

array([[ 935, 1773, 2070, 2788,  244,  594, 2882, 1126,  152, 2071, 2862,
        2343, 2342,  220,  221, 2124, 2123, 2832, 2584,  939, 1818, 2608,
        2462,  702,  935, 1773, 2070, 2788,  244,  594, 2882, 1126,  152,
        2071, 2862, 2343, 2342,  220,  221, 2124, 2123, 2832, 2584,  939,
        1818,  765,  702,  752, 1432, 1833, 2209, 1008, 1777, 2871, 1132,
        1008, 1006, 2905, 2902, 2680,  526, 1380, 2868, 2680]])

In [ ]:
initial_durations

array([[ 9,  9,  9, 12,  5,  8,  2,  9,  9,  9,  9,  5,  5,  8,  2,  5,
         5,  9,  9,  7,  3,  2,  4,  3,  9,  9,  9, 12,  5,  8,  2,  9,
         9,  9,  9,  5,  5,  8,  2,  5,  5,  9,  9,  7,  3,  2,  2,  1,
         3,  5,  5,  9,  9,  9,  9,  5,  5,  8,  2,  9,  9,  9,  9,  5]])

In [ ]:
# Function to predict chords and durations
def predict_chords(chord_sequence, duration_sequence,model,temperature=1.0):
    predicted_chords, predicted_durations = model.predict([chord_sequence, duration_sequence])
    return sample_with_temp(predicted_chords[0],temperature), sample_with_temp(predicted_durations[0],temperature)

In [ ]:
# Define empty lists for generated chords and durations
new_chords, new_durations = [], []

# Generate chords and durations using 120 rounds of prediction
for j in range(120):
    new_chord, new_duration = predict_chords(initial_chords, 
                                             initial_durations,
                                             model,
                                             temperature=0.8)
    new_chords.append(new_chord)
    new_durations.append(new_duration)
    initial_chords[0][:-1] = initial_chords[0][1:]
    initial_chords[0][-1] = new_chord
    initial_durations[0][:-1] = initial_durations[0][1:]
    initial_durations[0][-1] = new_duration

In [ ]:
# Create stream object
generated_stream = stream.Stream()
generated_stream.append(instrument.Piano())

# Add notes and durations to stream
for j in range(len(new_chords)):
    try:
        generated_stream.append(note.Note(int_to_note[new_chords[j]].replace('.', ' '), 
                                          quarterType = int_to_duration[new_durations[j]]))
    except:
        generated_stream.append(chord.Chord(int_to_note[new_chords[j]].replace('.', ' '), 
                                            quarterType = int_to_duration[new_durations[j]]))

# Export as MIDI file
generated_stream.write('midi', fp='lstm_120_rounds.mid')

'lstm_120_rounds.mid'